## Back Propagation Revision

Understanding _back propagation_ algorithm is really crucial. In this note, I try to implement it to make sure I comprehend the maths behind. This helps me to easily absorb other variations such as _back propagation through time (BPTT))_ in RNN.

___Objective:___
- Step 1: Compute the __gradient__ by back propogation.
- Step 2: Implement __numerical gradient__ and use it to check whether the gradient calculated in step 1 is correct

### 1. Compute Gradient using Back Propagation

Describe Neural Network:

- $L = 2$ --> 2 layers (2 hidden layers + 1 output layer)
- $N = 10$ --> 10 examples
- $D = H_0 = 15$ --> 15 input features
- $H_1 = 15$ --> 20 nodes in the 1st hidden layer)
- $C = H_2 = 3$ --> 3 node in the output layer, ie. 1 output classes)

Let's write some variables:

- $W^{(1)} \in \mathbf{R}^{D \times H_1}$
- $b^{(1)} \in \mathbf{R}^{1 \times H_1}$
- $z_j^{(l)} = W_{j:}^{(l)T} a^{(l-1)} + b_j^{(l)}$
- $z^{(l)} = W^{(l)T} a^{(l-1)} + b^{(l)}$. Note: $a^{(0)} = x$
- $x_i \in \mathbf{R}^{1 \times D}$ --> $X \in \mathbf{R}^{N \times D}$. Note: we usually use $x_i$ as a row vector in practice.
- $y_i \in \mathbf{R}^{1 \times C}$ --> $Y \in \mathbf{R}^{N \times C}$

In [1]:
import numpy as np
np.random.seed(1)

def my_rand(d1, d2):
    return np.random.rand(d1, d2) - 0.5

N = 10
D = 15
H1 = 20
C = 3

X = my_rand(N, D)
W1 = my_rand(D, H1)
W2 = my_rand(H1, C)

b1 = my_rand(1, H1)
b2 = my_rand(1, C)

Ws = [W1, W2]
bs = [b1, b2]

Y_truth = my_rand(N, C)

#### 1.1 Forward Propagation

In [2]:
# X: input
# Ws, bs: Weights and biases. Ws = [W1, W2, W3, ...]. bs = [b1, b2, b3, ...]
# activation_function: sigmoid, tanh, ReLU...
# Return: 
#   A = [a1, a2, a3, ...]. A[-1] is the predicted value (Y_predicted)
def forward_propagation(X, Ws, bs, activation):
    assert(len(Ws) == len(bs))
    
    Zs = []
    As = []
    for k in range(len(Ws)):
        a_prev = X if k == 0 else As[-1]
        zk = np.dot(a_prev, Ws[k]) + bs[k]
        ak = activation(zk)
        Zs.append(zk)
        As.append(ak)
    
    return As, Zs

In [3]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_gradient(x):
    z = sigmoid(x)
    return z * (1 - z)

# Use numerical gradient to check if sigmoid_gradient is correct
threshold = 1e-8
eps = 1e-6
m = 3
diff = sigmoid_gradient(m) - (sigmoid(m + eps) - sigmoid(m - eps))/(2*eps)

print('Check sigmoid_gradient by numerical gradient:\n')
print('diff:', diff, '\t--> Passed: ', diff < threshold)

Check sigmoid_gradient by numerical gradient:

diff: 3.99633590109e-11 	--> Passed:  True


#### 1.2 Back propagation

Gradients of cost function with respect to the weights of the $l^{th}$ layer:

$$\begin{eqnarray}
\frac{\partial J}{\partial w_{ij}^{(l)}} = \frac{\partial J}{\partial z_j^{(l)}}. \frac{\partial z_j^{(l)}}{\partial w_{ij}^{(l)}} = e_j^{(l)} a_i^{(l-1)}
\end{eqnarray}$$

where

$\begin{eqnarray}
e_j^{(l)} &=& \frac{\partial J}{\partial z_j^{(l)}} = \frac{\partial J}{\partial a_j^{(l)}} . \frac{\partial a_j^{(l)}}{\partial z_j^{(l)}} \\
&=& \left( \sum_{k = 1}^{d^{(l+1)}} \frac{\partial J}{\partial z_k^{(l+1)}} .\frac{\partial z_k^{(l+1)}}{\partial a_j^{(l)}} \right) f’(z_j^{(l)}) \\
 &=&\left( \sum_{k = 1}^{d^{(l+1)}} e_k^{(l+1)} w_{jk}^{(l+1)} \right) f’(z_j^{(l)}) \\
 &=&\left( w_{j:}^{(l+1)} e^{(l+1)} \right) f’(z_j^{(l)}) \\
\end{eqnarray}$

The term $e^{(L)}$ of the output layer depends on the cost function (look at the term $\frac{\partial J}{\partial a_j^{(L)}}$ in it). For example, given the cost function as mean squared error (MSE) and $N = 1$, then 
$\begin{eqnarray}
e_j^{(L)} = (a_j^{(L)} - y).f'(z_j^{(L)})
\end{eqnarray}$.

With the same reasoning, we have: 
$\begin{eqnarray}
\frac{\partial J}{\partial b_{j}^{(l)}} = e_j^{(l)}
\end{eqnarray}$

For batch (mini-batch) gradient descent, visit http://machinelearningcoban.com/2017/02/24/mlp/

In [4]:
# X: input
# Y_truth: labels 
# Ws: [W1, W2, W3, ...]
# bs: [b1, b2, b3, ...]
# activation: activation function
# activation_gradient: function to calculate the gradient of the activation function
# Return:
#   Delta_Ws = [Delta_W1, Delta_W2, Delta_W3, ...]
#   Delta_bs = [Delta_b1, Delta_b2, Delta_b3, ...]
def back_propagation(X, Y_truth, Ws, bs, activation, activation_gradient):
    N, C = Y_truth.shape
    As, Zs = forward_propagation(X, Ws, bs, activation)
    
    # Compute Es
    Es = []
    for k in range(len(Ws))[::-1]:
        if k == len(Ws) - 1:
            # NOTE: This computation of E_L is based on the assumption that the cost function is SSE
            Ek = (As[-1] - Y_truth) / N * activation_gradient(Zs[-1])
        else:
            Ek = np.dot(Es[-1], Ws[k+1].T) * activation_gradient(Zs[k])
        Es.append(Ek)
    Es = Es[::-1] # Reverse it
    
    # Compute Delta_Ws
    Delta_Ws = []
    for k in range(len(Ws))[::-1]:
        a_prev = X if k == 0 else As[k-1]
        Delta_Wk = np.dot(a_prev.T, Es[k])
        Delta_Ws.append(Delta_Wk)
    Delta_Ws = Delta_Ws[::-1]
    
    # Compute Delta_bs
    Delta_bs = []
    for k in range(len(bs)):
        Delta_bk = Es[k].sum(axis=0).reshape(1, -1)
        Delta_bs.append(Delta_bk)
    
    return Delta_Ws, Delta_bs

### 2. Compare with Numerical Gradient
Computing gradient is extremely important but potentially error-prone. To make sure it's correct, we will compare the result with that of numerical gradient.

$\begin{eqnarray}
f'(z) \approx \frac{f(z+\varepsilon) - f(z-\varepsilon)}{2\varepsilon}
\end{eqnarray}$

In [5]:
# X: input
# Y_truth: labels
# Ws, bs: Weights and biases. Ws = [W1, W2, W3, ...]. bs = [b1, b2, b3, ...]
# Return: cost (loss) of the prediction using SSE
def cost(X, Y_truth, Ws, bs, activation):
    As, _ = forward_propagation(X, Ws, bs, activation)
    Y = As[-1]
    N, C = Y_truth.shape
    return 1/(2*N) * np.sum((Y - Y_truth) ** 2)

def neighbourhood(W, i, j, eps):
    W_plus = W.copy()
    W_minus = W.copy()
    W_plus[i][j] += eps
    W_minus[i][j] -= eps
    return W_plus, W_minus

# X: input
# Y_truth: labels 
# Ws, bs: Weights and biases. Ws = [W1, W2, W3, ...]. bs = [b1, b2, b3, ...]
# cost: cost function
# Return: Delta_Ws, Delta_bs
#   Delta_Ws = [Delta_W1, Delta_W2, Delta_W3, ...]
#   Delta_bs = [Delta_b1, Delta_b2, Delta_b3, ...]
def numerical_gradient(X, Y_truth, Ws, bs, cost, activation):
    assert(X.shape[0] == Y_truth.shape[0])
    assert(len(Ws) == len(bs))
    
    eps = 1e-6
    Delta_Ws = []
    Delta_bs = []
    for k in range(len(Ws)):
        # Gradients for Ws
        Wk = Ws[k]
        Delta_Wk = np.zeros_like(Wk)
        for i in range(Wk.shape[0]):
            for j in range(Wk.shape[1]):
                Wk_plus, Wk_minus = neighbourhood(Wk, i, j, eps)
                
                cost_plus = cost(X, Y_truth, Ws[:k] + [Wk_plus] + Ws[k+1:], bs, activation)
                cost_minus = cost(X, Y_truth, Ws[:k] + [Wk_minus] + Ws[k+1:], bs, activation)
                Delta_Wk[i][j] = (cost_plus - cost_minus) / (2*eps)
                
        Delta_Ws.append(Delta_Wk)
    
        # Gradients for bs
        bk = bs[k]
        Delta_bk = np.zeros_like(bk)
        for i in range(bk.shape[0]):
            for j in range(bk.shape[1]):
                bk_plus, bk_minus = neighbourhood(bk, i, j, eps)
                
                cost_plus = cost(X, Y_truth, Ws, bs[:k] + [bk_plus] + bs[k+1:], activation)
                cost_minus = cost(X, Y_truth, Ws, bs[:k] + [bk_minus] + bs[k+1:], activation)
                Delta_bk[i][j] = (cost_plus - cost_minus) / (2*eps)
                
        Delta_bs.append(Delta_bk)

    return Delta_Ws, Delta_bs

In [6]:
# ------------------------------------
from numpy.linalg import linalg as LA

As = forward_propagation(X, Ws, bs, sigmoid)
Delta_Ws1, Delta_bs1 = back_propagation(X, Y_truth, Ws, bs, sigmoid, sigmoid_gradient)
Delta_Ws2, Delta_bs2 = numerical_gradient(X, Y_truth, Ws, bs, cost, sigmoid)

assert(len(Delta_Ws1) == len(Delta_Ws2))
assert(len(Delta_bs1) == len(Delta_bs2))

print('Compare with numerical gradient:\n')
threshold = 1e-8
for k in range(len(Delta_Ws1)):
    assert(Delta_Ws1[k].shape == Delta_Ws2[k].shape)
    assert(Delta_bs1[k].shape == Delta_bs2[k].shape)
    norm_diff_Wk = LA.norm(Delta_Ws1[k] - Delta_Ws2[k])
    norm_diff_bk = LA.norm(Delta_bs1[k] - Delta_bs2[k])
    print('diff(W%d):' % (k+1), norm_diff_Wk, '\t--> Passed:', norm_diff_Wk < threshold)
    print('diff(b%d):' % (k+1), norm_diff_bk, '\t--> Passed:', norm_diff_Wk < threshold)

Compare with numerical gradient:

diff(W1): 1.11115868661e-09 	--> Passed: True
diff(b1): 2.49699209548e-10 	--> Passed: True
diff(W2): 3.55458525023e-10 	--> Passed: True
diff(b2): 5.84080230485e-11 	--> Passed: True
